<a href="https://colab.research.google.com/github/FireStrings/MasterDegree/blob/main/SARIMA/Sarima_Runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import os
os.chdir("/content/drive/My Drive/UFSCAR/Notebooks")

Mounted at /content/drive/


#### Imports

In [ ]:
%run UtilsNew.ipynb

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from math import sqrt

In [ ]:
set_plot_size(15, 3)

In [ ]:
def apply_sarima(df, order, seasonal_order):
    model = SARIMAX(df, order=order, seasonal_order=seasonal_order, enforce_stationarity=False)

    model_fit = model.fit(trend="nc", disp=False) #Vericiar o que é o nc

    return model_fit

def predict(model_fit, df_to_model, col, start, end):
    df_to_model['forecast'] = model_fit.predict(start=start,end=end, dynamic=True)

def get_residual_with_metrics(df):

    df_residuos = df[~df["forecast"].isna()]
    df_residuos["diff"] = df_residuos["radiacao"] - df_residuos["forecast"]

    mse = mean_squared_error(df_residuos["radiacao"], df_residuos["forecast"])
    mae = mean_absolute_error(df_residuos["radiacao"], df_residuos["forecast"])
    r2 = r2_score(df_residuos["radiacao"], df_residuos["forecast"])

    return {"mse": mse, "rmse": sqrt(mse), "mae":mae, "r2":r2}

def apply_config_model(order, seasonal_order, results, station, config):

    df_gp = load_and_filter(station, 7, 18)[["radiacao"]]

    model_fit = apply_sarima(df_gp, order, seasonal_order)

    start_pred = int(len(df_gp) * 0.9)
    end_pred = len(df_gp)

    predict(model_fit, df_gp, "radiacao", start=start_pred-250, end=end_pred-250)
    # predict(model_fit, df_gp, "radiacao", start=start_pred, end=end_pred)
    metrics = get_residual_with_metrics(df_gp)

    results[station][config] = metrics

def process_sarima(results):
    lista_estacoes = ["campo_grande", "curitiba", "manaus", "minas_gerais_gv", "teresina", "pelotas", "brasilia", "minas_gerais_bh"]
    configs = {}
    configs[0] = [(1, 0, 0), (1, 0, 1, 12)]
    configs[1] = [(0, 0, 0), (1, 0, 1, 12)]
    configs[2] = [(1, 0, 1), (1, 0, 1, 12)]

    for estacao in lista_estacoes:
        print("Starting process for station " + estacao)
        results[estacao] = {}
        for config in configs.keys():
            apply_config_model(configs[config][0], configs[config][1], results, estacao, config)


In [ ]:
results = {}
process_sarima(results)

Starting process for station campo_grande
Starting process for station curitiba
Starting process for station manaus
Starting process for station minas_gerais_gv
Starting process for station teresina
Starting process for station pelotas
Starting process for station brasilia
Starting process for station minas_gerais_bh


In [ ]:
def create_table_results_sarima(results):
    records = []
    for cidade, valores in results.items():
        for idx, metricas in valores.items():
            record = {'cidade': cidade, 'config': idx}
            record.update(metricas)
            records.append(record)

    df = pd.DataFrame(records)
    df.set_index('cidade', inplace=True)

    return df

In [ ]:
pd.set_option('display.float_format', '{:.3f}'.format)
create_table_results_sarima(results)

,config,mse,rmse,mae,r2
cidade,,,,,
campo_grande,0,872401.540,934.024,747.453,0.268
campo_grande,1,1819004.012,1348.705,1045.620,-0.527
campo_grande,2,859929.285,927.324,744.630,0.278
curitiba,0,1906756.565,1380.854,1050.734,-0.984
curitiba,1,1700396.493,1303.993,978.172,-0.770
curitiba,2,1703629.019,1305.231,980.013,-0.773
manaus,0,1991403.170,1411.171,1088.692,-1.032
manaus,1,1445707.197,1202.376,902.939,-0.475
manaus,2,1796863.612,1340.471,1024.845,-0.833
